In [1]:
!pip install torch torchvision opencv-python scikit-image matplotlib tqdm



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\Kirtan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
# Clone the PIFuHD repository
!git clone https://github.com/facebookresearch/pifuhd.git


Cloning into 'pifuhd'...


d:\SEM 6\AI\Project\pifuhd
⏬ Downloading pifuhd.pt...


In [1]:
%cd pifuhd

d:\SEM 6\AI\Project\pifuhd


In [3]:
# Download the pretrained PIFuHD model directly using Python
import os
import urllib.request

os.makedirs("checkpoints", exist_ok=True)
url = "https://dl.fbaipublicfiles.com/pifuhd/checkpoints/pifuhd.pt"
output_path = "checkpoints/pifuhd.pt"

if not os.path.exists(output_path):
    print("⏬ Downloading pifuhd.pt...")
    urllib.request.urlretrieve(url, output_path)
    print("✅ Download complete!")
else:
    print("✔️ Already downloaded.")



⏬ Downloading pifuhd.pt...
✅ Download complete!


In [4]:
import cv2
from datetime import datetime

def capture_photo(output_dir="sample_images"):
    os.makedirs(output_dir, exist_ok=True)

    # Use webcam
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("❌ Could not open camera.")
        return None

    print("📸 Press SPACE to capture the photo, ESC to cancel...")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("❌ Failed to grab frame.")
            break

        cv2.imshow("Live Camera Feed", frame)
        key = cv2.waitKey(1)

        if key % 256 == 27:  # ESC to exit
            print("❌ Capture cancelled.")
            cap.release()
            cv2.destroyAllWindows()
            return None

        elif key % 256 == 32:  # SPACE to capture
            filename = f"{output_dir}/captured_{datetime.now().strftime('%Y%m%d_%H%M%S')}.jpg"
            cv2.imwrite(filename, frame)
            print(f"✅ Image captured and saved as: {filename}")
            cap.release()
            cv2.destroyAllWindows()
            return filename

# Example usage:
image_path = capture_photo()


📸 Press SPACE to capture the photo, ESC to cancel...
✅ Image captured and saved as: sample_images/captured_20250412_142012.jpg


In [8]:
!pip install models

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [22 lines of output]
      Traceback (most recent call last):
        File "C:\Users\Kirtan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "C:\Users\Kirtan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\Kirtan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_pro

In [7]:
import torch
import cv2
import numpy as np
from models.with_mobilenet import PoseEstimationWithMobileNet
from modules.keypoints import extract_keypoints, group_keypoints
from modules.load_state import load_state
from modules.pose import Pose, track_poses
import demo

def get_rect(net, images, height_size):
    net = net.eval()

    stride = 8
    upsample_ratio = 4
    num_keypoints = Pose.num_kpts
    previous_poses = []
    delay = 33
    for image in images:
        rect_path = image.replace('.%s' % (image.split('.')[-1]), '_rect.txt')
        img = cv2.imread(image, cv2.IMREAD_COLOR)
        orig_img = img.copy()
        orig_img = img.copy()
        heatmaps, pafs, scale, pad = demo.infer_fast(net, img, height_size, stride, upsample_ratio, cpu=False)

        total_keypoints_num = 0
        all_keypoints_by_type = []
        for kpt_idx in range(num_keypoints):  # 19th for bg
            total_keypoints_num += extract_keypoints(heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)

        pose_entries, all_keypoints = group_keypoints(all_keypoints_by_type, pafs)
        for kpt_id in range(all_keypoints.shape[0]):
            all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
            all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale
        current_poses = []

        rects = []
        for n in range(len(pose_entries)):
            if len(pose_entries[n]) == 0:
                continue
            pose_keypoints = np.ones((num_keypoints, 2), dtype=np.int32) * -1
            valid_keypoints = []
            for kpt_id in range(num_keypoints):
                if pose_entries[n][kpt_id] != -1.0:  # keypoint was found
                    pose_keypoints[kpt_id, 0] = int(all_keypoints[int(pose_entries[n][kpt_id]), 0])
                    pose_keypoints[kpt_id, 1] = int(all_keypoints[int(pose_entries[n][kpt_id]), 1])
                    valid_keypoints.append([pose_keypoints[kpt_id, 0], pose_keypoints[kpt_id, 1]])
            valid_keypoints = np.array(valid_keypoints)

            if pose_entries[n][10] != -1.0 or pose_entries[n][13] != -1.0:
              pmin = valid_keypoints.min(0)
              pmax = valid_keypoints.max(0)

              center = (0.5 * (pmax[:2] + pmin[:2])).astype(np.int_)  # Or np.int32
              radius = int(0.65 * max(pmax[0]-pmin[0], pmax[1]-pmin[1]))
            elif pose_entries[n][10] == -1.0 and pose_entries[n][13] == -1.0 and pose_entries[n][8] != -1.0 and pose_entries[n][11] != -1.0:
              # if leg is missing, use pelvis to get cropping
              center = (0.5 * (pose_keypoints[8] + pose_keypoints[11])).astype(np.int_)
              radius = int(1.45*np.sqrt(((center[None,:] - valid_keypoints)**2).sum(1)).max(0))
              center[1] += int(0.05*radius)
            else:
              center = np.array([img.shape[1]//2,img.shape[0]//2])
              radius = max(img.shape[1]//2,img.shape[0]//2)

            x1 = center[0] - radius
            y1 = center[1] - radius

            rects.append([x1, y1, 2*radius, 2*radius])

        np.savetxt(rect_path, np.array(rects), fmt='%d')

ModuleNotFoundError: No module named 'models'

In [ ]:
import numpy as np
import torch
import cv2
import os
from pathlib import Path


from models.with_mobilenet import PoseEstimationWithMobileNet
from modules.load_state import load_state

# Load pose estimation model
net = PoseEstimationWithMobileNet()
checkpoint_path = "./lightweight-human-pose-estimation.pytorch/checkpoint_iter_370000.pth"
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
load_state(net, checkpoint)

# Generate rect
get_rect(net, image_path)  # generates _rect.txt file


ModuleNotFoundError: No module named 'get_rect'